# Teacher Student Network Research

##### Teacher Student Network Research
Framework adapted from Official Pytorch Knowledge Distillation Tutorial

Author: 
Asad Amiruddin, 
Harrison Maximillian Rush, 
Huy N Ho

### Import library, datasets, loaders

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Below we are preprocessing data for CIFAR-10. We use an arbitrary batch size of 128.
transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
# Loading the CIFAR-10 dataset:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
 
# Dataloaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


### Define train and test function

In [30]:
def train(model, train_loader, epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def test(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy
 

### Define deeper neural networks to be used as teachers. 
Can have multiple teachers for comparison/experiments


### Load resnet50 model with finetuned weight as another teacher

In [ ]:
teacher_resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
teacher_resnet50.fc = nn.Linear(teacher_resnet50.fc.in_features, 10)
teacher_resnet50 = teacher_resnet50.to(device)
from google.colab import drive
drive.mount('/content/drive')  
teacher_resnet50.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/teacher_finetuned_cifar10.pth",map_location=device  )) 
# teacher_resnet50.load_state_dict(torch.load("./trained_model/teacher_finetuned_cifar10.pth",map_location=device)) 
test_accuracy_teacher = test(teacher_resnet50, test_loader, device)



### Define student network


In [25]:
# Define the student model
class studentNN(nn.Module):
    def __init__(self):
        super(studentNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 10)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x 

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Instantiate the model
# studentNN = studentNN().to(device)

# test_accuracy_student = test(studentNN, test_loader, device)

### Define knowledge distillation function

In [26]:

def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight, device):
    print('Knowledge distillation training')
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with the teacher model - do not save gradients here as we do not change the teacher's weights
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with the student model
            student_logits = student(inputs)

            #Soften the student logits by applying softmax first and log() second
            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft target loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

### Instantiate the 2 identical student nets

In [27]:

# Instantiate the lightweight network:
# We instantiate one more lightweight network model to compare their performances.
# Back propagation is sensitive to weight initialization,
# so we need to make sure these two networks have the exact same initialization.
torch.manual_seed(42)
learning_student = studentNN().to(device)

torch.manual_seed(42)
non_learning_student = studentNN().to(device)

# To ensure we have created a copy of the first network, we inspect the norm of its first layer.
# If it matches, then we are safe to conclude that the networks are indeed the same.
 
# Print the total number of parameters in each model:
total_params_teacher = "{:,}".format(sum(p.numel() for p in teacher_resnet50.parameters()))
print(f"DeepNN parameters: {total_params_teacher}")
total_params_non_learning_student = "{:,}".format(sum(p.numel() for p in non_learning_student.parameters()))
print(f"non_learning_student parameters: {total_params_non_learning_student}")
total_params_learning_student = "{:,}".format(sum(p.numel() for p in learning_student.parameters()))
print(f"learning_student parameters: {total_params_learning_student}")

Norm of 1st layer of nn_light: 2.327361822128296
Norm of 1st layer of new_nn_light: 2.327361822128296
DeepNN parameters: 23,528,522
non_learning_student parameters: 267,738
learning_student parameters: 267,738


### Train the student and compare to the one without teacher

In [28]:

# IMPORTANT: To load the pretrained model from either Google Drive/Local Drive instead of training it, set one of the booleans to True. 
# Make sure the .pth file exists in the respective directory
# .pth files are created further down after all training are completed 

load_non_learning_student_model_from_Gdrive = False   # need to be on Colab 
load_non_learning_student_model_from_local_hard_drive = False   

if load_non_learning_student_model_from_Gdrive:
    from google.colab import drive
    drive.mount('/content/drive')  
    non_learning_student.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/non_learning_student.pth"  )) 
elif load_non_learning_student_model_from_local_hard_drive : 
    non_learning_student.load_state_dict(torch.load("./trained_model/non_learning_student.pth",map_location=device))    
else:
    train(non_learning_student, train_loader, epochs=10, learning_rate=0.001, device=device)
 
test_accuracy_non_learning = test(non_learning_student, test_loader, device)
 
print(f"Student accuracy: {test_accuracy_non_learning:.2f}%")


load_learning_student_model_from_Gdrive = False   # need to be on Colab 
load_learning_student_model_from_local_hard_drive = False   

if load_learning_student_model_from_Gdrive:
    from google.colab import drive
    drive.mount('/content/drive')  
    learning_student.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/learning_student.pth"  )) 
elif load_learning_student_model_from_local_hard_drive : 
    learning_student.load_state_dict(torch.load("./trained_model/learning_student.pth",map_location=device))    
else:
    # Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
    train_knowledge_distillation(teacher=teacher, student=learning_student, 
                                 train_loader=train_loader, epochs=10, learning_rate=0.001, 
                                 T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)

test_accuracy_learning_student = test(learning_student, test_loader, device)

# Compare the student test accuracy with and without the teacher, after distillation
# Result shouldn't be stellar because teacher's prediction can't beat ground truth here
print(f"Teacher accuracy: {test_accuracy_teacher:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_non_learning:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_learning_student:.2f}%")

KeyboardInterrupt: 

###learning student

In [ ]:
load_learning_student_model_from_Gdrive = False   # need to be on Colab 
load_learning_student_model_from_local_hard_drive = False   

if load_learning_student_model_from_Gdrive:
    from google.colab import drive
    drive.mount('/content/drive')  
    learning_student.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/learning_student.pth"  )) 
elif load_learning_student_model_from_local_hard_drive : 
    learning_student.load_state_dict(torch.load("./trained_model/learning_student.pth",map_location=device))    
else:
    # Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
    train_knowledge_distillation(teacher=teacher_resnet50, student=learning_student, 
                                 train_loader=train_loader, epochs=10, learning_rate=0.001, 
                                 T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)

test_accuracy_learning_student = test(learning_student, test_loader, device)

# Compare the student test accuracy with and without the teacher, after distillation
# Result shouldn't be stellar because teacher's prediction can't beat ground truth here
print(f"Teacher accuracy: {test_accuracy_teacher:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_non_learning:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_learning_student:.2f}%")

### Save trained models - only run after training on Colab

In [61]:
# IMPORTANT
# change the boolean below to True to save the trained model .pth file into your Google Drive
# only work if executing on Google Colab
save_model_to_Gdrive = True # need to be on Colab 
save_model_to_local_drive = False 
if save_model_to_Gdrive:
    from google.colab import drive
    drive.mount('/content/drive')
    torch.save(teacher_resnet50.state_dict(), "/content/drive/MyDrive/Colab Notebooks/teacher_resnet50.pth")
    torch.save(non_learning_student.state_dict(), "/content/drive/MyDrive/Colab Notebooks/non_learning_student.pth") # non-learning student
    torch.save(learning_student.state_dict(), "/content/drive/MyDrive/Colab Notebooks/learning_student.pth") # student after KD
elif save_model_to_local_drive:
    torch.save(teacher_resnet50.state_dict(), "./trained_model/teacher_resnet50.pth")
    torch.save(non_learning_student.state_dict(), "./trained_model/non_learning_student.pth") # non-learning student
    torch.save(learning_student.state_dict(), "./trained_model/learning_student.pth") # student after KD
    
    

### sandbox section: train the learning student on "unlabeled" data/transfer learning/finetuning

In [64]:
 
!git clone https://github.com/pytorch/captum 

%cd captum

!git checkout "v0.2.0"

!pip3 install -e .

import sys
sys.path.append('/content/captum')

%cd ..



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001963F74BB50>
Traceback (most recent call last):
  File "C:\Users\huyho\DL\teacher-student-network\.venv\lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "C:\Users\huyho\DL\teacher-student-network\.venv\lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


KeyboardInterrupt: 

In [ ]:
import captum
import captum.attr as captum_attr
import matplotlib.pyplot as plt

layer = learning_student.layer4
# Create a LayerConductance object
lc = captum_attr.LayerConductance(learning_student, layer)

# Pass input data through the model
# Assuming 'input_data' is your input tensor
attributions = lc.attribute(input_data)

# Step 6: Visualize the attribution
# Assuming you want to visualize the attributions for a specific image in the batch
plt.imshow(attributions[0].cpu().detach().numpy(), cmap='viridis')
plt.show()
